# Step5.3: get_VITfeats

运行它获得vit feature，为train做准备

In [1]:
import json
from PIL import Image
import torch
from torchvision import transforms
from pytorch_pretrained_vit import ViT

In [2]:
import os
path = "./data/dataset_images_boxes"

if os.path.exists(path+"/.ipynb_checkpoints"):
    os.rmdir(path+"/.ipynb_checkpoints")
    print("rmdir .checkpoints")

imgs = os.listdir(path)
# imgs = imgs[:100]
print(len(imgs))

24635


In [3]:
#model = ViT('B_32_imagenet1k', pretrained=True)
model = ViT('B_32', pretrained=True)
print(model.image_size)
tfms = transforms.Compose([transforms.Resize(model.image_size), transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),])
#tfms = transforms.Compose([transforms.Resize(model.image_size), transforms.ToTensor(), transforms.Normalize(0.5, 0.5)])

RuntimeError: Error(s) in loading state_dict for ViT:
	size mismatch for fc.weight: copying a param with shape torch.Size([21843, 768]) from checkpoint, the shape in current model is torch.Size([768, 768]).
	size mismatch for fc.bias: copying a param with shape torch.Size([21843]) from checkpoint, the shape in current model is torch.Size([768]).

In [ ]:
import pickle
from tqdm import tqdm
model.eval()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
def generate_features_block(start, end, filename):
    print(f"now : [{start:8}, {end:8}, {filename:8}]")
    with torch.no_grad():
        dic = {}
        for idx in tqdm(range(start,end)):
            name = imgs[idx]
            dic[name] = dict()
            path_ = path + "/" + name
            if os.path.exists(f"{path_}/.ipynb_checkpoints"):
                os.system(f"rm -r {path_}/.ipynb_checkpoints")
            boxes = os.listdir(path_)
            for box in boxes:
                img = Image.open(f"{path_}/{box}")
                img = tfms(img).unsqueeze(0)
                img = img.to(device)

                # outputs = model(img).squeeze(0)
                outputs = model(img).squeeze(0)
                print(outputs.shape)
                dic[name][box[:-4]] = outputs.cpu()
        with open(f"/root/autodl-tmp/vit_features.B32.finetuned.{filename:02}.pkl",'wb') as fout:
            pickle.dump(dic,fout)

In [ ]:
def generate_features(num, block_size=1000):
    cnt = 0
    i = 0
    while i < num:
        if i+block_size < num:
            generate_features_block(i, i+block_size, cnt)
        else:
            generate_features_block(i, num, cnt)
        i += block_size
        cnt += 1

In [ ]:
generate_features(len(imgs), 2400)

`dic`的格式如下：


```
{
    '682716753374351360': {
        'green wall': tensor([-0.2158, -0.5861, -0.4583,  ..., -0.4584, -0.0507, -0.4340]), 
        'thumb': tensor([-0.0943, -0.5097, -0.2850,  ..., -0.2975,  0.4255,  0.7431]), 
        'yellow circle': tensor([-0.3630,  0.0496, -0.5468,  ..., -0.5510, -0.0705, -0.4504]), 
        'blue lights': tensor([-0.4361, -0.5627, -0.6589,  ..., -0.6676, -0.3880, -0.6415]), 
        'clear glass': tensor([-0.3609, -0.6426, -0.6136,  ..., -0.6126, -0.5325, -0.5997]), 
        'hand': tensor([-0.4882, -0.5795, -0.6744,  ..., -0.6797, -0.4926, -0.5218]), 
        'blue dot': tensor([-0.0286, -0.0159, -0.2335,  ..., -0.2459, -0.5378,  0.2008]), 
        'man': tensor([-0.4034, -0.9045, -0.6479,  ..., -0.6523, -0.0525, -0.8093]), 
        'white words': tensor([-0.4899,  0.1618, -0.7007,  ..., -0.7047, -0.7450, -0.6402])
    }, 
    '682721949072625664': {
        'logo': tensor([ 0.1612,  0.3313, -0.0857,  ..., -0.1012,  0.2761,  0.1852]), 
        'white logo': tensor([ 0.0115, -0.3251, -0.1830,  ..., -0.1912,  0.7098, -0.2490]), 
        'blue sky': tensor([-0.1040,  0.2694, -0.2814,  ..., -0.2938, -0.1567,  0.1455]), 
        'white dog': tensor([-0.1709,  0.2743, -0.3800,  ..., -0.3780,  0.4276, -0.6060]), 
        'picture': tensor([ 0.1025,  0.3527, -0.0880,  ..., -0.0990,  0.7861, -0.1227]), 
        'blue wall': tensor([ 0.0800,  0.5311, -0.1281,  ..., -0.1368,  0.2102, -0.1506]), 
        'black letter': tensor([-0.2343, -0.0206, -0.4577,  ..., -0.4666,  0.2765,  0.2039]), 
        'green wall': tensor([-0.3552, -0.3162, -0.5775,  ..., -0.5958, -0.9602, -0.4342])
    }, 
    '682722242111774720': {
    ...
    },
}
```